In [20]:
import pickle, time, os
import numpy as np
import paddle
import pandas as pd
import paddle.nn as nn
from tqdm import tqdm

from visualdl import LogWriter
logwriter = LogWriter(logdir='./runs')
# visualdl --logdir ./runs/ --host 0.0.0.0 --port 8040

users_df = pd.read_csv('data/csv/users.csv')
items_df = pd.read_csv('data/csv/items.csv')

In [21]:
emb_scale = 64
batch_size = 4096
len_users = len(users_df)
len_items = len(items_df)

In [22]:
class Dataset(paddle.io.Dataset):
    def __init__(self, data):
        self.data = data
        self.w = len(data[0])
        self.h = len(data)
        pass
    def __getitem__(self, idx):
        w = idx % self.w
        h = int(idx / self.h)
        return np.append(w, h), self.data[w][h]
    def __len__(self):
        return self.w * self.h
    pass 

In [23]:
class DSSM(nn.Layer):
    def __init__(self):
        super(DSSM,self).__init__()
        self.users_emb = nn.Embedding(len_users+1, emb_scale)
        self.items_emb = nn.Embedding(len_items+1, emb_scale)
        self.fc = nn.Linear(2 * emb_scale, 1)
        # self.users_fc1 = nn.Linear(emb_scale, 512)
        # self.users_fc2 = nn.Linear(512, 300)
        # self.users_fc3 = nn.Linear(300, 300)
        # self.users_fc4 = nn.Linear(300, 128)
        # self.items_fc1 = nn.Linear(emb_scale, 512)
        # self.items_fc2 = nn.Linear(512, 300)
        # self.items_fc3 = nn.Linear(300, 300)
        # self.items_fc4 = nn.Linear(300, 128)
        pass
    def forward(self, input):
        # user tower
        user = self.users_emb(input[:,0])
        # user = self.users_fc1(user)
        # user = nn.functional.relu(user)
        # user = self.users_fc2(user)
        # user = nn.functional.relu(user)
        # user = self.users_fc3(user)
        # user = nn.functional.relu(user)
        # user = self.users_fc4(user)
        # item tower
        item = self.items_emb(input[:,1])
        # item = self.items_fc1(item)
        # item = nn.functional.relu(item)
        # item = self.items_fc2(item)
        # item = nn.functional.relu(item)
        # item = self.items_fc3(item)
        # item = nn.functional.relu(item)
        # item = self.items_fc4(item)
        # similarity
        x = nn.functional.cosine_similarity(user, item, axis=1)
        x = nn.functional.sigmoid(x)
        return x
    pass
dssm = DSSM()
optim = paddle.optimizer.Adam(parameters=dssm.parameters(), learning_rate=0.005)
m = paddle.metric.Recall()

In [24]:
train_loss_, train_acc_, eval_loss_, eval_acc_ = [], [], [], []

for epoch_id in range(10):

    train_loss, train_acc, eval_loss, eval_acc = [], [], [], []

    file_list = os.listdir('data/net/train/')
    file = open('data/net/train/'+np.random.choice(file_list), 'rb')
    train_dataset = paddle.io.DataLoader(Dataset(pickle.load(file)),
                                         drop_last=True,
                                         batch_size=batch_size,
                                         shuffle=True)
    file.close()    

    dssm.train()
    m.reset()
    for batch_id, data in enumerate(tqdm(train_dataset)):
        x_data = data[0]
        y_data = data[1]
        x_data = paddle.cast(x_data, dtype='int32')

        y_pred = dssm(x_data)
        loss = nn.functional.mse_loss(y_pred, paddle.cast(y_data, dtype='float32'))
        acc = paddle.static.accuracy(paddle.reshape(y_pred, (batch_size, 1)), paddle.reshape(paddle.cast(y_data, dtype='int64'), (batch_size, 1)))
        m.update(y_pred, y_data)
        loss.backward()

        optim.step()
        optim.clear_grad()
        train_loss.append(loss.numpy())
        train_acc.append(acc.numpy())

        logwriter.add_scalar("train_loss", value=loss.numpy(), step=batch_id+epoch_id*(batch_size))
        logwriter.add_scalar("train_acc", value=acc.numpy(), step=batch_id+epoch_id*(batch_size))
        logwriter.add_scalar("train_recall", value=m.accumulate(), step=batch_id+epoch_id*(batch_size))

    file_list = os.listdir('data/net/eval/')
    file = open('data/net/eval/'+np.random.choice(file_list), 'rb')
    train_dataset = paddle.io.DataLoader(Dataset(pickle.load(file)),
                                         drop_last=True,
                                         batch_size=batch_size,
                                         shuffle=True)
    file.close() 

    dssm.eval()
    m.reset()
    for batch_id, data in enumerate(tqdm(train_dataset)):
        x_data = data[0]
        y_data = data[1]
        x_data = paddle.cast(x_data, dtype='int32')

        y_pred = dssm(x_data)
        loss = nn.functional.mse_loss(y_pred, paddle.cast(y_data, dtype='float32'))
        acc = paddle.static.accuracy(paddle.reshape(y_pred, (batch_size, 1)), paddle.reshape(paddle.cast(y_data, dtype='int64'), (batch_size, 1)))
        m.update(y_pred, y_data)

        logwriter.add_scalar("eval_loss", value=loss.numpy(), step=batch_id+epoch_id*(batch_size))
        logwriter.add_scalar("eval_acc", value=acc.numpy(), step=batch_id+epoch_id*(batch_size))
        logwriter.add_scalar("eval_recall", value=m.accumulate(), step=batch_id+epoch_id*(batch_size))

    print("epoch_id: {}, batch_id: {}, loss: {}, acc: {}, recall: {}".format(epoch_id, batch_id+1, loss.numpy(), acc.numpy(), m.accumulate()))

100%|██████████| 1258/1258 [03:10<00:00,  6.60it/s]


FileNotFoundError: [WinError 3] 系统找不到指定的路径。: 'data/net/test/'